In [1]:
from ase.build import fcc111

In [2]:
slab = fcc111('Al', size = (2,2,3), vacuum = 10.0)

In [3]:
from ase.visualize import view

In [7]:
from ase.io import write

In [8]:
write('al.cif', slab)

In [9]:
slab

Atoms(symbols='Al12', pbc=[True, True, False], cell=[[5.727564927611035, 0.0, 0.0], [2.8637824638055176, 4.960216729135935, 0.0], [0.0, 0.0, 24.676537180435968]], tags=...)

In [10]:
slab.cell

Cell([[5.727564927611035, 0.0, 0.0], [2.8637824638055176, 4.960216729135935, 0.0], [0.0, 0.0, 24.676537180435968]])

In [11]:
type(slab)

ase.atoms.Atoms

In [12]:
slab.positions

array([[ 1.43189123,  0.82670279, 10.        ],
       [ 4.2956737 ,  0.82670279, 10.        ],
       [ 2.86378246,  3.30681115, 10.        ],
       [ 5.72756493,  3.30681115, 10.        ],
       [ 0.        ,  1.65340558, 12.33826859],
       [ 2.86378246,  1.65340558, 12.33826859],
       [ 1.43189123,  4.13351394, 12.33826859],
       [ 4.2956737 ,  4.13351394, 12.33826859],
       [ 0.        ,  0.        , 14.67653718],
       [ 2.86378246,  0.        , 14.67653718],
       [ 1.43189123,  2.48010836, 14.67653718],
       [ 4.2956737 ,  2.48010836, 14.67653718]])

In [14]:
slab.cell[:]

array([[ 5.72756493,  0.        ,  0.        ],
       [ 2.86378246,  4.96021673,  0.        ],
       [ 0.        ,  0.        , 24.67653718]])

In [17]:
slab.get_scaled_positions()

array([[0.16666667, 0.16666667, 0.40524324],
       [0.66666667, 0.16666667, 0.40524324],
       [0.16666667, 0.66666667, 0.40524324],
       [0.66666667, 0.66666667, 0.40524324],
       [0.83333333, 0.33333333, 0.5       ],
       [0.33333333, 0.33333333, 0.5       ],
       [0.83333333, 0.83333333, 0.5       ],
       [0.33333333, 0.83333333, 0.5       ],
       [0.        , 0.        , 0.59475676],
       [0.5       , 0.        , 0.59475676],
       [0.        , 0.5       , 0.59475676],
       [0.5       , 0.5       , 0.59475676]])

In [19]:
slab.get_chemical_symbols()

['Al', 'Al', 'Al', 'Al', 'Al', 'Al', 'Al', 'Al', 'Al', 'Al', 'Al', 'Al']

In [42]:
def atomic_positions(file, slab):
    file.write('ATOMIC_POSITIONS (crystal) \n')
    
    names = slab.get_chemical_symbols()
    positions = slab.get_scaled_positions() 
    
    for i in range(len(names)):
        str_positions = ""
        for j in range(3):
            str_positions += str(positions[i][j]) + " "
        
        file.write(" " + names[i] + " " + str_positions + "\n")

In [43]:
f = open('test.txt', 'w')

atomic_positions(f, slab)

f.close()

In [48]:
A=slab.get_cell()[:]

In [49]:
A[0]

array([5.72756493, 0.        , 0.        ])

In [50]:
def crystal_parameters(file, slab):
    file.write('CRYSTAL_PARAMETERS (alat)\n')
    
    cell = slab.get_cell()[:]
    
    for i in range(3):
        str_lattice = ""
        for j in range(3):
            str_lattice += str(cell[i][j]) + " "
        
        file.write(" " + str_lattice + "\n")

In [51]:
f = open('test.txt', 'w')

crystal_parameters(f, slab)

f.close()

# Platinum (100)

In [55]:
pt_slab = fcc111('Pt', size = (2,2,3), vacuum = 10.0)

In [54]:
plat = open('pt.relax.in', 'w')

In [56]:
atomic_positions(plat, pt_slab)

In [57]:
crystal_parameters(plat, pt_slab)

In [58]:
plat.close()

In [59]:
write('pt.cif', pt_slab)

In [66]:
pt_slab = fcc111('Pt', size = (2,2,1), vacuum = 10.0)

In [67]:
write('pt.cif', pt_slab)

# ASE Tutorials

## Introduction: Nitrogen on Copper

In [2]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate

### Atoms
Atoms object is a collection of atoms. We can define molecules/structures by specifying the position of two atoms (nitrogen atoms). 

In [4]:
from ase import Atoms

In [5]:
d = 1.10 
molecule = Atoms('2N', positions = [(0.0, 0.0, 0.0), (0.0, 0.0, d)])

#### Can also build crystals using lattice module with common structures

In [6]:
from ase.build import fcc111
# fcc111 builds a fcc111 structures

In [7]:
slab = fcc111('Cu', size = (4,4,2), vacuum = 10.0)

### Including a Calculator
- Attach a calculator to an atom object
- Each atoms object has a `calc` attribute that we can set equal to a calculator such as EMT
- Then we can calculate the total energy using the `get_potential_energy()` method from the atoms class

In [8]:
from ase.calculators.emt import EMT

In [9]:
slab.calc = EMT()
molecule.calc = EMT()

In [10]:
energy_slab = slab.get_potential_energy()

In [11]:
energy_N2 = molecule.get_potential_energy()

### Structure Relaxation
1. We can consider the structure of N2 absorbed onto a Cu surface. 
    - We start by adding the absorbate to the slab (on top) 
    - Keep the Cu atoms fixed using `FixAtoms` from the `constraints` module. So we only allow for the N2 molecule to relax during the relaxation routine.
        - This involves defining a constraint and then adding the constraint to the structure
    - Add the `QuasiNewton` minimizer to the system and save the trajectory file. Then we run the minimizer with a constraint that `fmax = 0.05`

In [13]:
h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

In [15]:
from ase.constraints import FixAtoms

# we define a constraint variable/object using FixAtoms and the condition
# that atom symbols that are not nitrogen are held in place 

constraint = FixAtoms(mask = [a.symbol != 'N' for a in slab])

# then to the slab we add the constraint 
slab.set_constraint(constraint)

In [16]:
from ase.optimize import QuasiNewton

In [17]:
dyn = QuasiNewton(slab, trajectory = 'N2Cu.traj')

In [20]:
dyn.run(fmax = 0.0005)

BFGSLineSearch:    3[  6] 08:20:04       11.625228*       0.0309
BFGSLineSearch:    4[  7] 08:20:04       11.625212*       0.0039
BFGSLineSearch:    5[  8] 08:20:04       11.625212*       0.0000


True

## Input-Output

### Writing atomic positions out to a file
We can use the write function to write out file. In particular, from this, we can generate a quantum espresso input file with the cell structure already filled in which would be super convenient for us. For instance, if we call the following... 

In [24]:
from ase.io import write
write('slab.espresso-in', slab)

We end up with the following output file. So for our use case, we can
1. Generate a structure in ASE and check if it's correct via built in visualizer or by generating a `.cif` file and passing it to VESTA. 
2. Once verified, we can then generate a quantum espresso `.in` file to then use 

```Fortran
&CONTROL
/
&SYSTEM
   ntyp             = 2
   nat              = 34
   ibrav            = 0
/
&ELECTRONS
/
&IONS
/
&CELL
/

ATOMIC_SPECIES
Cu 63.546 None
N 14.007 None

K_POINTS gamma

CELL_PARAMETERS angstrom
10.21062192033375 0.00000000000000 0.00000000000000
5.10531096016687 8.84265797144727 0.00000000000000
0.00000000000000 0.00000000000000 22.08423447177455

ATOMIC_POSITIONS angstrom
Cu 0.0000000000 1.4737763286 10.0000000000  0 0 0
Cu 2.5526554801 1.4737763286 10.0000000000  0 0 0
Cu 5.1053109602 1.4737763286 10.0000000000  0 0 0
Cu 7.6579664403 1.4737763286 10.0000000000  0 0 0
Cu 1.2763277400 3.6844408214 10.0000000000  0 0 0
Cu 3.8289832201 3.6844408214 10.0000000000  0 0 0
Cu 6.3816387002 3.6844408214 10.0000000000  0 0 0
Cu 8.9342941803 3.6844408214 10.0000000000  0 0 0
Cu 2.5526554801 5.8951053143 10.0000000000  0 0 0
Cu 5.1053109602 5.8951053143 10.0000000000  0 0 0
Cu 7.6579664403 5.8951053143 10.0000000000  0 0 0
Cu 10.2106219203 5.8951053143 10.0000000000  0 0 0
Cu 3.8289832201 8.1057698072 10.0000000000  0 0 0
Cu 6.3816387002 8.1057698072 10.0000000000  0 0 0
Cu 8.9342941803 8.1057698072 10.0000000000  0 0 0
Cu 11.4869496604 8.1057698072 10.0000000000  0 0 0
Cu 0.0000000000 0.0000000000 12.0842344718  0 0 0
Cu 2.5526554801 0.0000000000 12.0842344718  0 0 0
Cu 5.1053109602 0.0000000000 12.0842344718  0 0 0
Cu 7.6579664403 0.0000000000 12.0842344718  0 0 0
Cu 1.2763277400 2.2106644929 12.0842344718  0 0 0
Cu 3.8289832201 2.2106644929 12.0842344718  0 0 0
Cu 6.3816387002 2.2106644929 12.0842344718  0 0 0
Cu 8.9342941803 2.2106644929 12.0842344718  0 0 0
Cu 2.5526554801 4.4213289857 12.0842344718  0 0 0
Cu 5.1053109602 4.4213289857 12.0842344718  0 0 0
Cu 7.6579664403 4.4213289857 12.0842344718  0 0 0
Cu 10.2106219203 4.4213289857 12.0842344718  0 0 0
Cu 3.8289832201 6.6319934786 12.0842344718  0 0 0
Cu 6.3816387002 6.6319934786 12.0842344718  0 0 0
Cu 8.9342941803 6.6319934786 12.0842344718  0 0 0
Cu 11.4869496604 6.6319934786 12.0842344718  0 0 0
N -0.0000000000 -0.0000000000 14.1420845717 
N -0.0000000000 0.0000000000 15.1889909183 


```

## Visualization
1. Simplest method: use the `view()` function which returns a 

In [25]:
from ase.visualize import view 
view(slab)

<Popen: returncode: None args: ['/Users/vinhtran/opt/anaconda3/bin/python', ...>

# Atoms and Calculators

## Atoms
1. Setting up a molecule and running a DFT calculation 
    - Can create simple molecules by manually typing the chemical symbols and specifying the atoms' positions in Angstrom 

In [28]:
from ase import Atoms
atoms = Atoms('N2', positions = [[0,0,-1] , [0,0,1]])

In [29]:
from ase.visualize import view

In [31]:
view(atoms)

<Popen: returncode: None args: ['/Users/vinhtran/opt/anaconda3/bin/python', ...>